# Import dependencies

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
import time
import mediapipe as mp
import os,sys
from sklearn.metrics import accuracy_score
import pickle
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
import pandas as pd
import csv

# Keypoints using MP Holistic

## Definitions

In [2]:
mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils

In [42]:
def mediapipe_detection(image, model):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image.flags.writeable = False
    results = model.process(image)
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    return image, results

In [50]:
def draw_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_TESSELATION)
    mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS)
    

def draw_styled_landmarks(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 1,circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 3,circle_radius=3)
                             )
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )

def draw_styled_landmarks_np(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 1,circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 1,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )

def draw_styled_landmarks_npf(image, results):
    mp_drawing.draw_landmarks(image,results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(0,0,255), thickness = 1,circle_radius=2),
                             mp_drawing.DrawingSpec(color=(255,255,255), thickness = 1,circle_radius=1)
                             )
    
    mp_drawing.draw_landmarks(image,results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )
    mp_drawing.draw_landmarks(image,results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(43,180,255), thickness = 2,circle_radius=3),
                             mp_drawing.DrawingSpec(color=(255,0,255), thickness = 2,circle_radius=1)
                             )

## Detection Testing

### Detecting from an image

In [7]:
#Enter picture input/ file path
rawpic = os.path.join("Guardian Demon 18.png")

#convert to a numpy array
pic = cv2.imread(rawpic)

In [15]:
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    
    #make detections
    picpr, results = mediapipe_detection(pic, holistic)
    
    #draw landmarks
    #draw_landmarks(image, results)
    draw_styled_landmarks(picpr, results)
    
    #display image
    plt.imshow(cv2.cvtColor(picpr, cv2.COLOR_BGR2RGB))
    plt.show()

error: OpenCV(4.5.5) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cv::cvtColor'


In [9]:
#Save output as jpeg
cv2.imwrite("out.jpg", picpr)

True

### Detecting from Webcam

#### Overlayed

##### laptop webcam

In [46]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)

        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

##### IP webcam

In [53]:
cap = cv2.VideoCapture("http://192.168.1.36:8080/video")

#Resizing feed
scale_percent = 40 # percent of original size
width = int(image.shape[1] * scale_percent / 100)
height = int(image.shape[0] * scale_percent / 100)
dim = (width, height)

#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)
        
        #Resizing
        #resized = cv2.resize(image, dim, interpolation = cv2.INTER_AREA)
        resized = cv2.resize(image, dim)
        
        #Flipping
        im_flip = cv2.rotate(resized,0)
        
        #show to screen
        cv2.imshow('OpenCV Feed', im_flip)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

#### Black background

In [12]:
#First large number is the number of pixels in the columns or width
frame1 = np.full((640,3),0)
# the *160 is the number of pixels in the rows/ height divided by 3
frame2 = np.array([frame1,frame1,frame1]*160,dtype=np.uint8)

cap = cv2.VideoCapture("http://192.168.43.52:8080/video")
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
        
        #style stuff, change frame2 to image from draw landmarks below for overlayed output
        frame2 = np.array([frame1,frame1,frame1]*160,dtype=np.uint8)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(frame2, results)

        #show to screen
        cv2.imshow('OpenCV Feed', frame2)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

### Detecting from a video

In [8]:
#Enter video input/ file path
video = "signtest.avi"

#### Overlayed

In [9]:
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Establish capture
    cap = cv2.VideoCapture(os.path.join(video))
    
    #Setup Video writer
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    #Videowriter
    video_writer = cv2.VideoWriter(os.path.join('output3.mp4'), cv2.VideoWriter_fourcc('M','P','4','2'), fps, (width, height))
    
    # Loop through each frame
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)

        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(image, results)

        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #Write out frame
        #video_writer.write(image)

        #Breaking the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    #close down everything
    cap.release()
    cv2.destroyAllWindows()
    video_writer.release()

#### Black Background

In [16]:
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    #Establish capture
    cap = cv2.VideoCapture(os.path.join(video))
    
    #Setup Video writer
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    fps = cap.get(cv2.CAP_PROP_FPS)
    
    frame1 = np.full((width,3),0)
        
    #Videowriter
    video_writer = cv2.VideoWriter(os.path.join('signttestOutput.mp4'), cv2.VideoWriter_fourcc('P','I','M','1'), fps, (width, height))
    
    # Loop through each frame
    for frame_idx in range(int(cap.get(cv2.CAP_PROP_FRAME_COUNT))):
        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
        
        frame2 = np.array([frame1,frame1,frame1]*int(height/3),dtype=np.uint8)

        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks(frame2, results)
        #draw_styled_landmarks_np(frame2, results)

        #show to screen
        cv2.imshow('OpenCV Feed', frame2)
        
        #Write out frame
        #video_writer.write(frame2)

        #Breaking the loop
        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

    #close down everything
    cap.release()
    cv2.destroyAllWindows()
    video_writer.release()

# Extracting keypoint values

## Creating CSV file to collect landmarks

In [26]:
import csv

In [27]:
face_f = np.zeros(468)
hand_f = np.zeros(21)
pose_f = np.zeros(33)

In [28]:
num_coords = len(hand_f) + len(face_f) + len(hand_f)

In [29]:
landmarks = ['class']
for val in range(1, num_coords+1):
    landmarks +=['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [30]:
#### DO NOT RERUN THIS IT CREATES AN EMPTY CSV FILE WITHOUT ANY DATA
with open('coordswf.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

## Defining class to collect

In [51]:
#Enter new classification class
class_name = "K"

## Keypoint collection from webcam

### With face data

In [34]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks_np(image, results)
        
        #Export Cordinates
        try:
            #Extract hand and face  landmarks
            lh_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3))
            rh_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3))
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3))
            
            #Concatenate rows
            row = lh_row + rh_row + face_row
            
            #Append Class Name
            row.insert(0,class_name)
            
            #Export to CSV
            with open('coords1.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            pass
        

        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

### Without face Data

In [52]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks_npf(image, results)
        
        #Export Cordinates
        try:
            #Extract hand and face  landmarks
            lh_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3))
            rh_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3))
            face_row = list(np.zeros(468*3))
            
            #Concatenate rows
            row = lh_row + rh_row + face_row
            
            #Append Class Name
            row.insert(0,class_name)
            
            #Export to CSV
            with open('coordswf.csv', mode='a', newline='') as f:
                csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                csv_writer.writerow(row)
            
        except:
            pass
        

        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

# Train Custom model with Scikit Learn

In [53]:
import pandas as pd
from sklearn.model_selection import train_test_split

In [54]:
df = pd.read_csv('coordswf.csv')

In [55]:
df.head()

,class,x1,y1,z1,x2,y2,z2,x3,y3,z3,...,z507,x508,y508,z508,x509,y509,z509,x510,y510,z510
0,Hello,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,Hello,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,Hello,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,Hello,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,Hello,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [56]:
X = df.drop('class', axis=1) #features
y = df['class'] #target value

In [57]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

## Train Machine learning Classification Model

In [58]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [59]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression()),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier()),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier()),
}

In [60]:
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\linear_model\_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [61]:
fit_models

{'lr': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('logisticregression', LogisticRegression())]),
 'rc': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('ridgeclassifier', RidgeClassifier())]),
 'rf': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('randomforestclassifier', RandomForestClassifier())]),
 'gb': Pipeline(steps=[('standardscaler', StandardScaler()),
                 ('gradientboostingclassifier', GradientBoostingClassifier())])}

## Evaluate and Serialize Model

In [62]:
fit_models['rf'].predict(X_test)

array(['K', 'M', 'A', ..., 'Hello', 'M', 'M'], dtype=object)

In [63]:
from sklearn.metrics import accuracy_score
import pickle

In [64]:
for algo,model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))

lr 1.0
rc 0.9991438356164384
rf 0.9991438356164384
gb 1.0


In [65]:
fit_models['rf'].predict(X_test)

array(['K', 'M', 'A', ..., 'Hello', 'M', 'M'], dtype=object)

In [62]:
y_test

537     hello
1361       my
1943       my
2073       my
3048        A
        ...  
4602        K
4510        K
850      name
3387        A
1370       my
Name: class, Length: 1626, dtype: object

## Saving Model

In [66]:
with open('signsHellomynameisNarrownf.pkl', 'wb') as f:
    pickle.dump(fit_models['rf'], f)

## Make Detections

In [67]:
with open('signsHellomynameisNarrownf.pkl', 'rb') as f:
    model = pickle.load(f)

In [68]:
cap = cv2.VideoCapture(0)
#set mediapipe model
with mp_holistic.Holistic(min_detection_confidence=0.5, min_tracking_confidence=0.5) as holistic:
    while cap.isOpened():

        #read frame
        ret, frame = cap.read()

        #make detections
        image, results = mediapipe_detection(frame, holistic)
               
        #draw landmarks
        #draw_landmarks(image, results)
        draw_styled_landmarks_np(image, results)
        
        #Export Cordinates
        try:
            #Extract hand and face  landmarks
            lh_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.left_hand_landmarks.landmark]).flatten() if results.left_hand_landmarks else np.zeros(21*3))
            rh_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.right_hand_landmarks.landmark]).flatten() if results.right_hand_landmarks else np.zeros(21*3))
            face_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in results.face_landmarks.landmark]).flatten() if results.face_landmarks else np.zeros(468*3))
            
            #Concatenate rows
            row = lh_row + rh_row + face_row
            
            #Make Detections
            X = pd.DataFrame([row])
            sign_class = model.predict(X)[0]
            sign_prob = model.predict_proba(X)[0]
            print(sign_class, sign_prob)
            
            #Grab ear coords
            coords = tuple(np.multiply(
                            np.array(
                                (results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].x,
                                 results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_EAR].y)
                            ), [640,480]).astype(int))
            
            cv2.rectangle(image,
                          (coords[0], coords[1]+5),
                          (coords[0]+len(sign_class)*20, coords[1]-30),
                          (245, 117, 16), -1)
            cv2.putText(image, sign_class, coords,
                       cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            #Get status box
            cv2.rectangle(image, (0,0), (250,60), (245,117,16), -1)
            
            #Display probability
            cv2.putText(image, 'PROB'
                       , (15,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, str(round(sign_prob[np.argmax(sign_prob)],2))
                       , (10,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
            
            #Display class
            cv2.putText(image, 'CLASS'
                       , (95,12), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 0), 1, cv2.LINE_AA)
            cv2.putText(image, sign_class.split(' ')[0]
                       , (90,40), cv2.FONT_HERSHEY_SIMPLEX, 1, (255,255,255), 2, cv2.LINE_AA)
            
        except:
            pass
        
        #show to screen
        cv2.imshow('OpenCV Feed', image)
        
        #break gracefully
        if cv2.waitKey(10) & 0xFF ==ord('q'):
            break
    cap.release()
    cv2.destroyAllWindows()

C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Hello [0.05 0.52 0.   0.   0.   0.43 0.  ]


C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Hello [0.05 0.52 0.   0.   0.   0.43 0.  ]


C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


...

Hello [0.05 0.52 0.   0.   0.   0.43 0.  ]


C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Hello [0.05 0.52 0.   0.   0.   0.43 0.  ]


C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(
C:\ProgramData\Anaconda3\envs\playground\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


Hello [0.05 0.52 0.   0.   0.   0.43 0.  ]
